In [18]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

# Feature engineering function from before (no PCA, with interactions and weighted props)
def engineer_features_no_pca(df):
    df = df.copy()
    new_cols = {}

    # Weighted component-property features
    for comp in range(1, 6):
        for prop in range(1, 11):
            prop_col = f'Component{comp}_Property{prop}'
            frac_col = f'Component{comp}_fraction'
            new_cols[f'{prop_col}_weighted'] = df[prop_col] * df[frac_col]

    # Interaction features between fractions
    for i in range(1, 6):
        frac_i = f'Component{i}_fraction'
        for j in range(i + 1, 6):
            frac_j = f'Component{j}_fraction'
            new_cols[f'{frac_i}_x_{frac_j}'] = df[frac_i] * df[frac_j]
            new_cols[f'{frac_i}_div_{frac_j}'] = df[frac_i] / (df[frac_j] + 1e-8)

    # Ratios to total fraction
    total_fraction = sum(df[f'Component{i}_fraction'] for i in range(1, 6))
    new_cols['Total_fraction'] = total_fraction
    for i in range(1, 6):
        frac_col = f'Component{i}_fraction'
        new_cols[f'{frac_col}_to_total'] = df[frac_col] / (total_fraction + 1e-8)

    # Aggregate stats for component properties
    for comp in range(1, 6):
        props = [f'Component{comp}_Property{p}' for p in range(1, 11)]
        new_cols[f'Component{comp}_prop_mean'] = df[props].mean(axis=1)
        new_cols[f'Component{comp}_prop_std'] = df[props].std(axis=1)
        new_cols[f'Component{comp}_prop_min'] = df[props].min(axis=1)
        new_cols[f'Component{comp}_prop_max'] = df[props].max(axis=1)
        new_cols[f'Component{comp}_prop_range'] = new_cols[f'Component{comp}_prop_max'] - new_cols[f'Component{comp}_prop_min']

    # Polynomial features for component fractions (squares)
    for i in range(1, 6):
        frac_col = f'Component{i}_fraction'
        new_cols[f'{frac_col}_squared'] = df[frac_col] ** 2

    # Cross product polynomial features
    for i in range(1, 6):
        frac_i = f'Component{i}_fraction'
        for j in range(i + 1, 6):
            frac_j = f'Component{j}_fraction'
            new_cols[f'{frac_i}_x_{frac_j}'] = df[frac_i] * df[frac_j]

    # Combine original df with new columns
    new_features_df = pd.DataFrame(new_cols, index=df.index)
    df = pd.concat([df, new_features_df], axis=1)

    return df

# 1. Load data
X = pd.read_csv('/Users/xDAyN/Desktop/cs project/Project Shell.ai Hackathon/dataset/X_train.csv')
y = pd.read_csv('/Users/xDAyN/Desktop/cs project/Project Shell.ai Hackathon/dataset/y_train.csv')
X_test = pd.read_csv('/Users/xDAyN/Desktop/cs project/Project Shell.ai Hackathon/dataset/X_test.csv')

# 2. Feature engineering
X_fe = engineer_features_no_pca(X)
X_test_fe = engineer_features_no_pca(X_test)

# 3. Train-validation split
X_train, X_val, y_train, y_val = train_test_split(X_fe, y, test_size=0.2, random_state=42)

# 4. Train model for target 1 only (index 0)
target_index = 0
print(f"Training model for target {target_index + 1} / {y.shape[1]}")

model = xgb.XGBRegressor(
    max_depth=3,
    n_estimators=1000,
    learning_rate=0.05,
    reg_alpha=0.95,
    reg_lambda=5.1,
    eval_metric='mae',
    random_state=42,
    verbosity=1,
    early_stopping_rounds = 25
)

model.fit(
    X_train, y_train.iloc[:, target_index],
    eval_set=[(X_val, y_val.iloc[:, target_index])],
    verbose=50
)

# 5. Predict on train and validation sets
train_preds = model.predict(X_train)
val_preds = model.predict(X_val)

# ... [your original code here unchanged] ...

# 6. Evaluate
train_mape = mean_absolute_percentage_error(y_train.iloc[:, target_index], train_preds)
val_mape = mean_absolute_percentage_error(y_val.iloc[:, target_index], val_preds)

print(f"\nTrain MAPE for target {target_index + 1}: {train_mape:.4f}")
print(f"Validation MAPE for target {target_index + 1}: {val_mape:.4f}")

# --- NEW: Show feature importance by total gain ---
importance_dict = model.get_booster().get_score(importance_type='gain')
importance_df = (
    pd.DataFrame({
        'feature': list(importance_dict.keys()),
        'importance': list(importance_dict.values())
    })
    .sort_values(by='importance', ascending=False)
    .reset_index(drop=True)
)

print("\nTop 20 most important features by total gain:")
print(importance_df.head(20))





Training model for target 1 / 10
[0]	validation_0-mae:0.77284
[50]	validation_0-mae:0.33231
[100]	validation_0-mae:0.23385
[150]	validation_0-mae:0.19465
[200]	validation_0-mae:0.17358
[250]	validation_0-mae:0.15999
[300]	validation_0-mae:0.15103
[350]	validation_0-mae:0.14422
[400]	validation_0-mae:0.14003
[450]	validation_0-mae:0.13743
[500]	validation_0-mae:0.13550
[550]	validation_0-mae:0.13403
[600]	validation_0-mae:0.13317
[650]	validation_0-mae:0.13250
[700]	validation_0-mae:0.13193
[750]	validation_0-mae:0.13147
[800]	validation_0-mae:0.13098
[850]	validation_0-mae:0.13053
[870]	validation_0-mae:0.13052

Train MAPE for target 1: 0.2759
Validation MAPE for target 1: 0.6180

Top 20 most important features by total gain:
                                        feature  importance
0   Component2_fraction_div_Component5_fraction   21.788336
1                           Component5_fraction   11.120496
2   Component1_fraction_div_Component5_fraction    7.787182
3     Component4_fractio